## MNIST Image Classification 
Copyright: Zhibo Zhang, 2020

[Link to the colab file](https://colab.research.google.com/drive/17wE_50l9h7LGsmr6OL3m5AeXMS8UVzmx?usp=sharing)



In [0]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
import numpy as np

Load the train and the test data in batches, and preprocess them


In [0]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()

train_x = train_x.reshape((60000, 28, 28, 1))
test_x = test_x.reshape((10000, 28, 28, 1))

train_x = tf.cast(train_x, tf.float32)
test_x = tf.cast(test_x, tf.float32)
train_y = tf.cast(train_y, tf.float32)
test_y = tf.cast(test_y, tf.float32)

train_y = to_categorical(train_y, 10)
test_y = to_categorical(test_y, 10)

Build the ResNet Model with CNN for classification

In [0]:
class ConvModel(tf.keras.Model):

    def __init__(self):
        super(ConvModel, self).__init__()
        self.Conv1 = layers.Conv2D(
                filters=3, kernel_size=5, strides=(1, 1), 
                padding="same")

        self.Conv2 = layers.Conv2D(
                filters=64, kernel_size=5, strides=(1, 1), 
                padding="same", activation="relu")
        self.MaxPool2 = layers.MaxPool2D(pool_size=(2, 2))

        self.Conv3 = layers.Conv2D(
                filters=128, kernel_size=5, strides=(1, 1), 
                padding="same", activation="relu")
        self.MaxPool3 = layers.MaxPool2D(pool_size=(2, 2))
        self.batchnorm = layers.BatchNormalization()

        self.flatten = layers.Flatten()
        self.fully_conn1 = layers.Dense(256, activation='relu')
        self.fully_conn2 = layers.Dense(10, activation='sigmoid')

    def call(self, x):       
        block1 = self.Conv1(x)
        block2 = self.MaxPool2(self.Conv2(block1))
        block3 = self.batchnorm(self.MaxPool3(self.Conv3(block2)))
        
        flatten = self.flatten(block3)
        fl1 = self.fully_conn1(flatten)
        output = self.fully_conn2(fl1)

        return output

Define a loss (objective) function and the optimizer:

In [0]:
objective = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

Initialize and compile the model:

In [0]:
model = ConvModel()
model.compile(optimizer=optimizer, loss=objective, metrics=["accuracy"])

Start training:

In [217]:
model.fit(
    x=train_x,
    y=train_y,
    batch_size=128,
    epochs=20
)

Epoch 1/20
469/469 [==============================] - 4s 8ms/step - loss: 0.1723 - accuracy: 0.9402
Epoch 2/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0376 - accuracy: 0.9885
Epoch 3/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0240 - accuracy: 0.9922
Epoch 4/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0193 - accuracy: 0.9939
Epoch 5/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0146 - accuracy: 0.9953
Epoch 6/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0136 - accuracy: 0.9955
Epoch 7/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0147 - accuracy: 0.9954
Epoch 8/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0114 - accuracy: 0.9968
Epoch 9/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0091 - accuracy: 0.9973
Epoch 10/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0096 - accuracy: 0.9972

Evaluate the model on the test set:

In [218]:
loss, acc = model.evaluate(
    x=test_x,
    y=test_y,
    batch_size=128
)

79/79 [==============================] - 0s 3ms/step - loss: 0.0421 - accuracy: 0.9929
